# Assignment  week 2

## Notebook made by   (If not filled in correctly: 0 pts for assignment)

__Name(s)__: Adriaan de Vries, Verna Dankers

__Student id(s)__ : 10795227, 10761225

__Email(s)__: adriaan.devries@student.uva.nl, verna.dankers@student.uva.nl

### Pledge (taken from [Coursera's Honor Code](https://www.coursera.org/about/terms/honorcode) )



Put here a selfie with your photo where you hold a signed paper with the following text: (if this is team work, put two selfies here). The link must be to some place on the web, not to a local file. **Assignments without the selfies will not be graded and receive 0 points.**

> My answers to homework, quizzes and exams will be my own work (except for assignments that explicitly permit collaboration).

>I will not make solutions to homework, quizzes or exams available to anyone else. This includes both solutions written by me, as well as any official solutions provided by the course staff.

>I will not engage in any other activities that will dishonestly improve my results or dishonestly improve/hurt the results of others.

<img width=30% src='http://oi63.tinypic.com/2q3srh3.jpg'/>
<img src="http://i63.tinypic.com/2itj1ic.jpg" border="0">

## Questions

#### Book

* Chap 6: 6.8 to 6.13, 6.14 to 6.17. 
 
#### Programming

* [MakeInvertedIndex](MakeInvertedIndex1516.html): Exercises 8.1 to 8.6 about Boolean Search Engines and Exercises 9.1 to 9.5 about Ranked Retrieval 
 


# Book

#### 6.8 

$idf_t = log( \frac{N}{df_t} )$

For $df_t > 0$, the $idf_t$ will always stay between the lower and the upper bound. The lower bound for $idf_t$ will be reached when the $df_t = N$ and the upper bound will be reached when $df_t = 1$. So the $idf_t$ can only be one of the numbers from the finite set of the number between these two bounds.

#### 6.9 

The $idf_t$ of a term that appears in every document: $log(\frac{N}{N}) = log(1) = 0$

Such a word will act like a stop word when the $tf-idf$ weighting will be used, since its weight becomes zero. It will not be able to affect the search results, due to its zero weight.

#### 6.10

$tf idf_{t,d} = tf_{t,d} \times idf_t$

##### car
$tf idf_{car,d1} = 27 \times log(\frac{3}{3}) = 0 $  

$tf idf_{car,d2} = 4 \times log(\frac{3}{3}) = 0 $  

$tf idf_{car,d3} = 24 \times log(\frac{3}{3}) = 0 $  

##### auto

$tf idf_{auto,d1} = 3 \times log(\frac{3}{2}) = 0.53 $  

$tf idf_{auto,d2} = 33 \times log(\frac{3}{2}) = 5.81 $

$tf idf_{auto,d3} = 0 \times log(\frac{3}{2}) = 0 $  

##### insurance

$tf idf_{insurance,d1} = 0 \times log(\frac{3}{2}) = 0 $  

$tf idf_{insurance,d2} = 33 \times log(\frac{3}{2}) = 5.81 $  

$tf idf_{insurance,d3} = 29 \times log(\frac{3}{2}) = 5.11 $  

##### best

$tf idf_{best,d1} = 14 \times log(\frac{3}{2}) = 2.47 $  

$tf idf_{best,d2} = 0 \times log(\frac{3}{2}) = 0 $  

$tf idf_{best,d3} = 17 \times log(\frac{3}{2}) = 2.99 $  


#### 6.11

Yes, obivously. Dependend on the number of documents in the collection (N) the upper bound for $tf-idf$ is $log(\frac{N}{1})$, which can definitely exceed one. This is illustrated by exercise 6.10. 

#### 6.12

The base of the logarithm will affect the score calculation, by lowering the $idf$ value when using a larger number for the base with the same $N$ and $df$. $log_{n+1}(\frac{N}{df})$ will always result in a lower number than $log_{n}(\frac{N}{df})$. Therefore the $tf idf$ score will be lower as well.

When comparing relative scores of different documents, they will seem more alike when a higher number has been used for the base of the log in the $idf$ calculation. Similarly they will seem less alike when a lower number has been used for the base of the log.

Increasing the number for the base "compresses" the $idf$ and therefore the $tf-idf$.



#### 6.13

#### 6.14

When applying stemming the terms being indexed are stemmed before indexing. Jealous and jealousy would be considered as a single dimension. Then:

$tf_{jealous, d} :=  tf_{jealous,d} + tf_{jealousy,d}$

$idf_{jealous} := idf_{jealous} + idf_{jealousy}$

ADRIAAN HOE KUNNEN WE DIT GENERALISEREN?

#### 6.15

Vectors from exercise 6.10:

1. car: $[0,0,0]^T$
2. auto: $[0.53,5.81,0]^T$ 
3. insurance: $[0,5.81,5.11]^T$ 
4. best:$[2.47,0,2.99]^T$

Normalized vectors:

1. car: We can not normalize the zero vector.
2. auto: $\frac{[0.53,5.81,0]^T}{\sqrt{0.53^2 + 5.81^2}} = [0.09, 0.997, 0]^T$
3. insurance: $\frac{[0,5.81,5.11]^T}{\sqrt{5.81^2 + 5.11^2}} = [0, 0.75, 0.66]^T$ 
4. best: $\frac{[2.47,0,2.99]^T}{\sqrt{2.47^2 + 2.99^2}} = [0.64, 0, 0.77]^T$

# Programming

For the programming part, we'll first copy and paste what we need from Assignment 1.

In [1]:
from bs4 import BeautifulSoup
from nltk.tokenize import WhitespaceTokenizer
from collections import Counter, defaultdict
import os
import pickle
import re

CORPUS = "Shakespeare"

def normalize(word, keep_capitals):
    """
    Normalize capital letters and punctuation
    """
    if (keep_capitals and not word == word.upper()) or not keep_capitals:
        word = word.lower()
    word = re.sub(r'[^\w\s]', '', word)
    return word

def prepare_data(folder, new_folder, keep_capitals, byte):
    """
    Prepare a text corpus for the process of
    making an inverted index.
    """
    # Create a directory for the processed files
    if not os.path.exists(new_folder):
        os.makedirs(new_folder)

    # Prepare each file one by one
    for filename in os.listdir(folder):
        if not os.path.exists(os.path.join(new_folder, filename[:-4] + ".pkl")):
            # Strip xml tags
            if byte:
                xmlfile = BeautifulSoup(open(os.path.join(folder, filename), 'rb'),"xml")
            else:
                xmlfile = BeautifulSoup(open(os.path.join(folder, filename), 'r'),"xml")
            textfile = xmlfile.get_text()

            # Split the text on whitespaces
            beautifile = WhitespaceTokenizer().tokenize(textfile)

            # Normalize text for capital letters, non-alphabetical tokens and whitespace
            finalfile = [ normalize(word, keep_capitals)
                          for word 
                          in beautifile 
                         ]
            new_filename = os.path.join(new_folder, filename[:-4] + ".pkl")
            savefile = open(new_filename, 'wb')
            pickle.dump(finalfile, savefile)


In [3]:
def index_collection(folder):
    """
    Create an inverted index given a folder 
    containig a text corpus.
    """
    index= defaultdict(Counter) # initialize MyIndex
    for filename in os.listdir(folder):
        text = pickle.load(open(os.path.join(folder,filename),'rb'))
        
        # Update the index with each token
        for w in text:    
            index[w][filename[:-4]]+=1   
    if '' in index.keys():
        del index['']
    return index

In [34]:
prepare_data(CORPUS, CORPUS + "New", 0, 0)
index_org = index_collection(CORPUS + "New")

In [6]:
def calculate_doc_freq(index, word):
    """
    Calculate in how many documents a given 
    term occurs, based on the inverted index.
    """
    return len(index[word])
    
def calculate_corp_freq(index, word):
    """
    Calculate how often a term occurs in a 
    corpus, based on the inverted index.
    """
    corpus_frequency = 0
    
    for document in index[word]:
        corpus_frequency += index[word][document]
        
    return corpus_frequency

def add_frequencies(index):
    for word in index:
        original_counter = index[word]
        document_frequency = calculate_doc_freq(index, word)
        corpus_frequency = calculate_corp_freq(index, word)
        index[word] = {"counter" : original_counter, 
                       "doc_freq" : document_frequency, 
                       "corp_freq" : corpus_frequency
                      }
    return index

In [35]:
index_with_freq = add_frequencies(index_org)

#### Making a list of all documents

Will be useful for negation.

In [57]:

def make_doc_set(index_with_frequencies):
    
    doc_list = []
    for token in index_with_freq:
        doc_list.extend(list(index_with_freq[token]['counter'].elements()))
    return set(doc_list)

In [59]:
doc_set = make_doc_set(index_with_freq)
print(doc_set)

{'tempest', 'lear', 'two_gent', 'hamlet', 'rich_iii', 'r_and_j', 'hen_v', 'troilus', 'cymbelin', 'all_well', 'lll', 't_night', 'taming', 'com_err', 'hen_vi_2', 'm_wives', 'm_for_m', 'john', 'timon', 'as_you', 'hen_viii', 'titus', 'hen_vi_1', 'win_tale', 'dream', 'pericles', 'a_and_c', 'hen_vi_3', 'rich_ii', 'merchant', 'j_caesar', 'much_ado', 'hen_iv_2', 'coriolan', 'othello', 'macbeth', 'hen_iv_1'}


### On to the actual exercises:

##### 8.1, a search engine for negation (so useful)

In [66]:
def docs_with_token(token):
    if token in index_with_freq:
        return set(index_with_freq[token]['counter'].elements())
    else:
        return set()

In [67]:
def negation_search(word):
    return doc_set.difference(docs_with_token(word))

In [72]:
print (negation_search('test'))
print (docs_with_token('test'))
print (negation_search('despair'))

{'lear', 'two_gent', 'rich_iii', 'r_and_j', 'hen_v', 'troilus', 'cymbelin', 'all_well', 'lll', 't_night', 'taming', 'com_err', 'hen_vi_2', 'm_wives', 'john', 'timon', 'as_you', 'hen_viii', 'titus', 'hen_vi_1', 'win_tale', 'dream', 'pericles', 'a_and_c', 'hen_vi_3', 'rich_ii', 'merchant', 'j_caesar', 'much_ado', 'hen_iv_2', 'coriolan', 'macbeth', 'hen_iv_1'}
{'tempest', 'hamlet', 'othello', 'm_for_m'}
{'hen_viii', 'pericles', 'two_gent', 'lll', 't_night', 'hamlet', 'm_wives', 'hen_v', 'troilus', 'j_caesar', 'much_ado', 'timon', 'othello', 'as_you', 'dream', 'hen_iv_1'}


#### 8.2 Disjunction search

Input as a list of words. Output is unordered.

In [76]:
def disjunction_search(query):
    if len(query) ==0:
        return set()
    result = docs_with_token(query[0])
    for word in query[1:]:
        result = result.union(docs_with_token(query[1]))
    return result

In [77]:
disjunction_search(['test', 'despair'])

{'a_and_c',
 'all_well',
 'com_err',
 'coriolan',
 'cymbelin',
 'hamlet',
 'hen_iv_2',
 'hen_vi_1',
 'hen_vi_2',
 'hen_vi_3',
 'john',
 'lear',
 'm_for_m',
 'macbeth',
 'merchant',
 'othello',
 'r_and_j',
 'rich_ii',
 'rich_iii',
 'taming',
 'tempest',
 'titus',
 'win_tale'}

##### 8.4 Conjunctive search

Input is, again, a list of words.

In [78]:
def conjunction_search(query):
    if len(query) ==0:
        return set()
    result = docs_with_token(query[0])
    for word in query[1:]:
        result = result.intersection(docs_with_token(query[1]))
    return result

In [79]:
print(conjunction_search(['test', 'despair']))

{'tempest', 'm_for_m'}


##### 8.5 Ordering of conjunctive search

Conjunctive search could be ordered by the amount of times terms appear in a document. Simply taking the sum is an option, or one could choose to do it relative to the rarity of the tokens. So the ranking could be the total amount of times a word from the query appears in the document, divided by the total amount of times the word appears in the corpus. Taking the sum over all the words in the query in this way would give a fairly good ranking.

Each document score could then possibibly be divided by the document length, to punish longer documents as they will obviously on average have more terms in them. One could, however, argue that these longer documents are better documents, so they should be ranked higher and not punished.

##### 8.6 Relaxing the search

Instead of removing documents that fail to contain a search word, we can add some form of counter to them. Only when the counter exceeds a specified value will it be deleted. Then what's left is to decide how many terms the document should at least match. We think three quarters would be a good threshold, so it would not be allowed to miss more than floor(N/4) where N is the amount of words in the query. This is easily adjustable to any other value.

As for ranking, the documents could be ranked the same way as specified in 8.5, except the documents with the lowest counter would be placed highest. So the documents that match the most terms will end up at the top of the ranking.